In [1]:
%run '/content/drive/MyDrive/Master_Machine_Learning/COVID19/models.ipynb'

In [2]:
import os
import json
from datetime import datetime
import torch
import time
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from tqdm import tqdm
import pandas as pd

# from models import resnet50	#

import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import label_binarize
from PIL import ImageFile
from sklearn.metrics import f1_score,precision_score,recall_score,roc_auc_score,accuracy_score

In [4]:
import os
import json
from datetime import datetime
import torch
import time
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from tqdm import tqdm
import pandas as pd

#from models import resnet50	#

import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import label_binarize
from PIL import ImageFile
from sklearn.metrics import f1_score,precision_score,recall_score,roc_auc_score,accuracy_score

def main():
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    # os.environ['CUDA_VISIBLE_DEVICES'] = '0,3'
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using {} device.".format(device))

    # 定义画图函数
    def matplot_loss(train_loss, val_loss,save_path):
        plt.plot(train_loss, label='train_loss')
        plt.plot(val_loss, label='val_loss')
        plt.legend(loc='best')
        plt.ylabel('loss-acc')
        plt.xlabel('epoch')
        plt.title("Accuracy-Loss")

        plt.savefig(save_path+'/'+'Loss.pdf',dpi=500,format='pdf')
        plt.savefig(save_path+'/'+'Loss.svg',dpi=500,bbox_inches = 'tight')
        plt.show()
    def matplot_acc(train_acc, val_acc,save_path):
        plt.plot(train_acc, label='train_acc')
        plt.plot(val_acc, label='val_acc')
        plt.legend(loc='best')
        plt.ylabel('loss-acc')
        plt.xlabel('epoch')
        plt.title("Accuracy-Loss")

        plt.savefig(save_path+'/'+'Accuracy.pdf',dpi=500,format='pdf')
        plt.savefig(save_path+'/'+'Accuracy.svg',dpi=500,bbox_inches = 'tight')
        plt.show()
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    #normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    data_transform = {
        "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     normalize,
                                     ]),
        "val": transforms.Compose([transforms.Resize(256),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   normalize
                                   ])}





    train_data = r'/content/drive/MyDrive/Dataset/covid/train'
    val_data = r'/content/drive/MyDrive/Dataset/covid/val'


    train_dataset = datasets.ImageFolder(train_data,
                                         transform=data_transform["train"])

    train_num = len(train_dataset)


    classes_list = train_dataset.class_to_idx
    cla_dict = dict((val, key) for key, val in classes_list.items())
    # write dict into json file
    json_str = json.dumps(cla_dict, indent=4)
    with open('class_indicesskin.json', 'w') as json_file:
        json_file.write(json_str)

    batch_size = 32
    nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 16])  # number of workers
    print('Using {} dataloader workers every process'.format(nw))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size, shuffle=True,
                                               num_workers=nw)

    validate_dataset = datasets.ImageFolder(val_data,
                                            transform=data_transform["val"])
    val_num = len(validate_dataset)
    validate_loader = torch.utils.data.DataLoader(validate_dataset,
                                                  batch_size=batch_size, shuffle=True,
                                                  num_workers=nw)

    print("using {} images for training, {} images for validation.".format(train_num,
                                                                           val_num))

    net = resnet50(num_classes=2)
    # pre_weights=torch.load("/home/lsw/projecet/utils/resnet50-19c8e357.pth")
    # pre_dict = {k: v for k, v in pre_weights.items() if net.state_dict()[k].numel() == v.numel()}
    # missing_keys, unexpected_keys = net.load_state_dict(pre_dict, strict=False)

    # change fc layer structure
    # if torch.cuda.device_count() > 1:
    #     net = nn.DataParallel(net)
    net.to(device)

    # define loss function
    # weight = torch.FloatTensor([6., 1., 13., 21., 6., 58., 47.])
    loss_function = nn.CrossEntropyLoss()
    # loss_function = nn.CrossEntropyLoss(weight=torch.from_numpy(np.array([0.44,0.73,0.83])).float(),reduction='mean')

    # loss_function.to(device)


    # construct an optimizer
    params = [p for p in net.parameters() if p.requires_grad]
    optimizer = optim.Adam(params, lr=0.0001)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    epochs = 300
    train_best_acc = 0.0
    val_best_acc = 0.0

    train_steps = len(train_loader)
    val_steps = len(validate_loader)
    loss_train = []
    acc_train = []
    loss_val = []
    acc_val = []
    for epoch in range(epochs):
        # train
        net.train()
        acc0 = 0.0
        running_loss = 0.0
        predlist=[]
        scorelist=[]
        targetlist=[]
        train_bar = tqdm(train_loader)
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()
            logits = net(images.to(device))

            loss = loss_function(logits, labels.to(device))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            predict_y0 = torch.max(logits, dim=1)[1]
            acc0 += torch.eq(predict_y0, labels.to(device)).sum().item()
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,epochs,loss)

        train_loss = running_loss / train_steps
        train_accurate = acc0 / train_num

        # validate
        net.eval()

        """acc = 0.0  # accumulate accurate number / epoch
        valdata_loss = 0.0
        time1 = "%s"%datetime.now()
        labels = [0,1]
        #labels = [0,1,2,3,4,5,6]
        with torch.no_grad():
            val_bar = tqdm(validate_loader)
            for val_data in val_bar:
                val_images, val_labels = val_data

                outputs = net(val_images.to(device))
                # print(outputs)

                predict_y = torch.max(outputs, dim=1)[1]
                # print(predict_y,val_labels)
                # exit()
                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()
                valloss = loss_function(outputs, val_labels.to(device))
                valdata_loss += valloss.item()
                val_bar.desc = "valid epoch[{}/{}]".format(epoch + 1,epochs)
                predlist=np.append(predlist, predict_y.cpu().numpy())
                targetlist=np.append(targetlist,val_labels)
        val_loss = valdata_loss / val_steps
        val_accurate = acc / val_num

        #用于绘图
        loss_train.append(train_loss)
        acc_train.append(train_accurate)
        loss_val.append(val_loss)
        acc_val.append(val_accurate)
        train_val_folder = r'/code/new/model_train_val/covid'
        if not os.path.exists(train_val_folder):
            os.mkdir(train_val_folder)
        list = [time1,epoch,train_loss,train_accurate,val_loss,val_accurate]
        data = pd.DataFrame([list])
        data.to_csv(train_val_folder+'/'+'train_log.csv',mode='w',header=False,index=False)
        output = "%s：Step [%d],  train_loss : %f, train_accuracy :  %g,val_loss : %f, val_accuracy :  %g" % (datetime.now(),epoch, train_loss, train_accurate,val_loss,val_accurate)
        with open(train_val_folder+'/'+'train_log.txt',"a+") as f:
            f.write(output+'\n')
            f.close
        with open(train_val_folder+'/'+'draw_lists.txt',"w") as f:
            f.write('{:}\n {:}\n {:}\n {:}\n '.format(loss_train,acc_train,loss_val,acc_val))
            f.close

        print('[epoch %d] train_loss: %.3f train_acc: %.3f val_loss: %.3f val_acc: %.3f ' %
              (epoch + 1, running_loss / train_steps,train_accurate,val_loss ,val_accurate))

        model_folder = r'/code/new/save_model/covid'
        if not os.path.exists(model_folder):
            os.mkdir(model_folder)
        save_path0 = model_folder+'/train.pth'
        save_path1 = model_folder+'/val.pth'
        if train_accurate > train_best_acc:
            train_best_acc = train_accurate
            torch.save(net.state_dict(),save_path0)
        if val_accurate > val_best_acc:
            val_best_acc = val_accurate
            torch.save(net.state_dict(),save_path1)
            train_val_folder = r'/code/new/model_train_val/covid'
            if not os.path.exists(train_val_folder):
                os.mkdir(train_val_folder)
            with open(train_val_folder+'/targetlist.txt','a+') as f:
                for tar in targetlist:
                    f.write(str(tar)+'\n')
                    f.close
            with open(train_val_folder+'/predlist.txt','a+') as f:
                for pre in predlist:
                    f.write(str(pre)+'\n')
                    f.close """

        acc = accuracy_score(targetlist, predlist, normalize=True)
        F1 = f1_score(targetlist, predlist, average='macro')
        precision = precision_score(targetlist, predlist,  labels=None, pos_label=1, average='macro')
        recall = recall_score(targetlist, predlist,  labels=None,average='macro', sample_weight=None)
        val_lab = label_binarize(targetlist, classes=labels)
        val_pre = label_binarize(predlist, classes=labels)
        auc = roc_auc_score(val_lab, val_pre, average='macro')
        print('acc',acc)
        print('F1',F1)
        print('precision',precision)
        print('recall',recall)
        print('auc',auc)
        print('best_acc',val_best_acc)
        with open(train_val_folder+'/metrics.txt','a+') as f:
            f.write('acc:{}, F1:{}, precision:{}, recall:{}, auc:{}\n'.format(acc,F1,precision,recall,auc))
            f.close
    output_folder = r'/code/new/output/covid'
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)

    matplot_loss(loss_train, loss_val,output_folder)
    matplot_acc(acc_train, acc_val,output_folder)
    print('Finished Training')




In [ ]:
if __name__ == '__main__':
    np.random.seed(0)
    torch.manual_seed(0)
    torch.cuda.manual_seed_all(0)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    main()

using cpu device.
Using 2 dataloader workers every process
using 35 images for training, 15 images for validation.


  0%|          | 0/2 [00:00<?, ?it/s]